In [1]:
from rag_assistant.rag_assistant import RAGAssistant
from langchain_classic.retrievers import ParentDocumentRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
import re

In [11]:
ParentDocumentRetriever?

Init signature:
ParentDocumentRetriever(
    *args: Any,
    name: str | None = None,
    tags: list[str] | None = None,
    metadata: dict[str, typing.Any] | None = None,
    vectorstore: langchain_core.vectorstores.base.VectorStore,
    byte_store: Optional[langchain_core.stores.BaseStore[str, bytes]] = None,
    docstore: langchain_core.stores.BaseStore[str, langchain_core.documents.base.Document],
    id_key: str = 'doc_id',
    search_kwargs: dict = <factory>,
    search_type: langchain_classic.retrievers.multi_vector.SearchType = <SearchType.similarity: 'similarity'>,
    child_splitter: langchain_text_splitters.base.TextSplitter,
    parent_splitter: langchain_text_splitters.base.TextSplitter | None = None,
    child_metadata_fields: collections.abc.Sequence[str] | None = None,
) -> None
Docstring:     
Retrieve small chunks then retrieve their parent documents.

When splitting documents for retrieval, there are often conflicting desires:

1. You may want to have small documents

In [2]:
assistant = RAGAssistant(persist_path="./chroma/rag", collection_name="blueprint_text_analytics", topic="Blueprints for Text Analytics in Python textbook")

Device being used in the embedding model is cpu

RAG Assistant initialized successfully for topic: Blueprints for Text Analytics in Python textbook


In [3]:
vector_db = assistant.vector_db

In [4]:
results = vector_db.similarity_search_with_score(query="Summarize chapter 1 for me")

In [5]:
results

[(Document(id='118dbb8b-b75a-43b1-a718-9db9988cd3ca', metadata={'Subtopic': '**Table of Contents**'}, page_content='Using Clustering to Uncover the Structure of Text Data                      236 Further Ideas                                                                                                                   240 Summary and Recommendation                                                                                 240 Conclusion                                                                                                                      241 **9. Text Summarization. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  243** What You’ll Learn and What We’ll Build                                                                   243 Text Summarization'),
  0.49578702449798584),
 (Document(id='c11e862c-2713-441b-9999-0ecf3bf75260', metadata={'Subtopic': '**Blueprint: Summarizing Text Using an Indicator Represen

In [14]:
results = vector_db.search(query="Summarize chapter 1 for me", search_type='mmr', k=5)

In [17]:
new_results = assistant._format_docs_with_metadata(results)

In [ ]:
with open("data/structured_text.md", "r", encoding='utf-8') as f:
    structured_text = f.read()

In [27]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[("#", "Chapter"), ("##", "Topic"), ("###", "Blueprint")],
    strip_headers=False
    )

In [38]:
markdown_content = markdown_splitter.split_text(structured_text)

In [168]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1200,
        chunk_overlap = 150
    )

In [128]:
docs = text_splitter.split_documents(markdown_content)